In [18]:
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

df_train=pd.read_csv('final_train.csv', encoding='utf-8', encoding_errors='ignore')
df_train.columns=['sentence','tags']

df_test=pd.read_csv('final_test.csv', encoding='utf-8', encoding_errors='ignore')
df_test.columns=['sentence','tags']

train_dataset=[]
valid_dataset=[]

train_texts = df_train.sentence.values.tolist()
train_labels= df_train.tags.values.tolist()

test_texts = df_test.sentence.values.tolist()
test_labels= df_test.tags.values.tolist()

new_train_texts = []
for train_texts in train_texts:
    new_train_texts.append(train_texts.split())
train_texts=new_train_texts

new_train_labels = []
for train_labels in train_labels:
    new_train_labels.append(train_labels.split())
train_labels=new_train_labels

new_test_texts = []
for test_texts in test_texts:
    new_test_texts.append(test_texts.split())
test_texts=new_test_texts

new_test_labels = []
for test_labels in test_labels:
    new_test_labels.append(test_labels.split())
test_labels=new_test_labels

for i in range(len(train_texts)):
    temp=[]
    for j in range(len(train_texts[i])):
        temp.append((train_texts[i][j],train_labels[i][j]))
    train_dataset.append(temp)
    
for i in range(len(test_texts)):
    temp=[]
    for j in range(len(test_texts[i])):
        temp.append((test_texts[i][j],test_labels[i][j]))
    valid_dataset.append(temp)
    
    

# Define a feature function to extract features from the input sentence
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

# Define feature extraction function for the entire sentence
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# Define function to extract labels from the dataset
def sent2labels(sent):
    return [label for token, label in sent]

# Extract features and labels from the training and validation sets
X_train = [sent2features(s) for s in train_dataset]
y_train = [sent2labels(s) for s in train_dataset]
X_valid = [sent2features(s) for s in valid_dataset]
y_valid = [sent2labels(s) for s in valid_dataset]

# Train a CRF model using the training set
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=False)
crf.fit(X_train, y_train)

# Evaluate the model using the validation set
y_pred = crf.predict(X_valid)
print(metrics.flat_classification_report(y_valid, y_pred))



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

          DF       1.00      1.00      1.00       196
        NAME       1.00      1.00      1.00      3643
           O       1.00      1.00      1.00      3660
    QUANTITY       1.00      1.00      1.00      2152
        SIZE       1.00      1.00      1.00       103
       STATE       1.00      1.00      1.00      1092
        TEMP       1.00      1.00      1.00        43
        UNIT       1.00      1.00      1.00      1777

    accuracy                           1.00     12666
   macro avg       1.00      1.00      1.00     12666
weighted avg       1.00      1.00      1.00     12666

